In [2]:
!pip install gdown

In [3]:
import gdown

file_id = "1uvSVd-9T2eDvaFK-PvxWEFSgiwmm9-3Q"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, output="my_file.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1uvSVd-9T2eDvaFK-PvxWEFSgiwmm9-3Q
From (redirected): https://drive.google.com/uc?id=1uvSVd-9T2eDvaFK-PvxWEFSgiwmm9-3Q&confirm=t&uuid=1a990d4f-3a98-41b6-a8ec-049488e89b99
To: /kaggle/working/my_file.zip
100%|██████████| 4.09G/4.09G [01:05<00:00, 62.5MB/s]


'my_file.zip'

In [4]:
import zipfile
import os

def extract_zip(zip_path, extract_to='.'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"✅ Extracted '{zip_path}' to '{os.path.abspath(extract_to)}'")


zip_file = "/kaggle/working/my_file.zip"  
extract_folder = "/kaggle/working/"  
os.makedirs(extract_folder, exist_ok=True)

extract_zip(zip_file, extract_folder)

✅ Extracted '/kaggle/working/my_file.zip' to '/kaggle/working'


In [5]:
import numpy as np
import pandas as pd

for dirname, _, filenames in os.walk('/kaggle/working'):
    print(dirname)

/kaggle/working
/kaggle/working/Dataset
/kaggle/working/Dataset/Train
/kaggle/working/Dataset/Train/Blocked
/kaggle/working/Dataset/Train/Normal
/kaggle/working/Dataset/Test
/kaggle/working/Dataset/Test/Blocked
/kaggle/working/Dataset/Test/Normal
/kaggle/working/__MACOSX
/kaggle/working/__MACOSX/Dataset
/kaggle/working/__MACOSX/Dataset/Train
/kaggle/working/__MACOSX/Dataset/Train/Blocked
/kaggle/working/__MACOSX/Dataset/Train/Normal
/kaggle/working/__MACOSX/Dataset/Test
/kaggle/working/__MACOSX/Dataset/Test/Blocked
/kaggle/working/__MACOSX/Dataset/Test/Normal
/kaggle/working/.virtual_documents


In [6]:
import torch 
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

In [7]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dir = "/kaggle/working/Dataset/Train"
test_dir = "/kaggle/working/Dataset/Test"

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset info
print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))
print("Classes:", train_dataset.classes)

Train samples: 12000
Test samples: 2000
Classes: ['Blocked', 'Normal']


In [8]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=2, f=32, k=3, u=256, r=0.5):
        super(CustomCNN, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=f, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=f, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=f, out_channels=f, kernel_size=k, padding=1),
            nn.BatchNorm2d(f),
            nn.MaxPool2d(kernel_size=2),
        )

        self.flatten = nn.Sequential(nn.Flatten(),nn.Linear(f * 8 * 8, 512))
        

        self.classifier = nn.Sequential( 
            nn.Linear(512, 256),# assuming input image size is 128x128
            nn.ReLU(),
            nn.BatchNorm1d(u),
            nn.Dropout(r),
            nn.Linear(u, num_classes)
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.flatten(x)
        conv_output = x
        x = self.classifier(x)
        return x,conv_output

In [9]:
from torchsummary import summary
import torch

model = CustomCNN().to("cuda")
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 128, 128]             896
       BatchNorm2d-2         [-1, 32, 128, 128]              64
         MaxPool2d-3           [-1, 32, 64, 64]               0
            Conv2d-4           [-1, 32, 64, 64]           9,248
       BatchNorm2d-5           [-1, 32, 64, 64]              64
         MaxPool2d-6           [-1, 32, 32, 32]               0
            Conv2d-7           [-1, 32, 32, 32]           9,248
       BatchNorm2d-8           [-1, 32, 32, 32]              64
         MaxPool2d-9           [-1, 32, 16, 16]               0
           Conv2d-10           [-1, 32, 16, 16]           9,248
      BatchNorm2d-11           [-1, 32, 16, 16]              64
        MaxPool2d-12             [-1, 32, 8, 8]               0
          Flatten-13                 [-1, 2048]               0
           Linear-14                  [

In [12]:
# Training function
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.to('cuda')
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            conv_output = outputs[1]
            outputs = outputs[0]
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        losses.append(running_loss/len(train_loader))
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
        print(conv_output.shape)

# Testing and classification report
def test_model(model, test_loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            outputs = outputs[0]
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

In [13]:
model = CustomCNN(num_classes=len(train_dataset.classes)).to('cuda')

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
losses = []

# Train and evaluate
train_model(model, train_loader, criterion, optimizer, "cuda", epochs=10)
test_model(model, test_loader, "cuda", class_names=train_dataset.classes)

Epoch [1/10], Loss: 0.2189
torch.Size([32, 512])
Epoch [2/10], Loss: 0.1690
torch.Size([32, 512])
Epoch [3/10], Loss: 0.1201
torch.Size([32, 512])
Epoch [4/10], Loss: 0.0992
torch.Size([32, 512])
Epoch [5/10], Loss: 0.0855
torch.Size([32, 512])
Epoch [6/10], Loss: 0.0781
torch.Size([32, 512])
Epoch [7/10], Loss: 0.0928
torch.Size([32, 512])
Epoch [8/10], Loss: 0.0694
torch.Size([32, 512])
Epoch [9/10], Loss: 0.0614
torch.Size([32, 512])
Epoch [10/10], Loss: 0.0568
torch.Size([32, 512])

Classification Report:
              precision    recall  f1-score   support

     Blocked       0.98      0.92      0.95      1000
      Normal       0.93      0.98      0.95      1000

    accuracy                           0.95      2000
   macro avg       0.95      0.95      0.95      2000
weighted avg       0.95      0.95      0.95      2000



In [15]:
from IPython.display import FileLink
torch.save(model.state_dict(), "student-MSE.pth")
FileLink("student-MSE.pth")

/kaggle/working/student-MSE.pth